In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

loc = "angular_distr_el/CH9/ID_ALL_mol_e0_valid/EN_gate/MFPADs_multinew_std"

fileRCR = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CR_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
fileRCL = uproot.open(r"D:\UniFRK\SEXTANT_sept2018\R-C3H3F3O_550eV_CL_9600-3700ns_newEfield_multiCH9_MFPAD_30_t0.root")
k=fileRCR[loc].keys()
d=dict(fileRCR[loc].classes()) # shows the type of each item
dict(fileRCR[loc]) # keys : values
check = fileRCR[loc+"/MFPAD3D_engate_costheta_-1.00_phi_-180"]
# type(check)
# check._members()

In [ ]:
MFPAD_RCR=[] # initilialises a list
ctetha_RCR=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCR[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm))
            cosphi.append((ctheta_nm+0.1666,phi_nm+15)) # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCR[filename].numpy())
            MFPAD_RCR.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCR[filename].numpy())
            ctetha_RCR.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCR=np.array(MFPAD_RCR)
ctetha_RCR=np.array(ctetha_RCR) # it is a list!

In [ ]:
MFPAD_RCL=[] # initilialises a list
ctetha_RCL=[]
cosphi=[]
MFPAD_xy=[]
run_one=0
for key, value in fileRCL[loc].items():
        filename=loc+"/"+str(key).split(";")[0].replace("b'","")
        if "mfpad3d_engate_costheta" in filename.lower():
            # print(filename)
            ctheta_nm=float((str(key).split("costheta_")[1]).split("_phi")[0])
            phi_nm=float((str(key).split("phi_")[1]).split(";")[0])
            # phicos.append((ctheta_nm,phi_nm)) #they are inverted
            cosphi.append((ctheta_nm+0.1666,phi_nm+15))  # centered in the middle of the bins
            #ctheta_n=re.search("costheta_(.*)_phi", str(key)).group(1) #alternative with import re
            temp=np.array(fileRCL[filename].numpy())
            MFPAD_RCL.append(temp[0]) # it is a list!
            if run_one == 0:
                MFPAD_xy.append((temp[1][0][0] , temp[1][0][1])) # phi cos(theta) from 2D
                run_one=1
        elif "cos(theta)" in filename.lower():
            ctheta_nc=float((str(key).split("costheta_")[1]).split("_phi")[0])
            temp=np.array(fileRCL[filename].numpy())
            ctetha_RCL.append(temp[0]) # it is a list!
            ctheta_c = temp[1] # cos(theta) from 1D
MFPAD_RCL=np.array(MFPAD_RCL)
ctetha_RCL=np.array(ctetha_RCL) # it is a list!

In [ ]:
plt.contourf(cosphi[:,1].reshape(12,6),cosphi[:,0].reshape(12,6),MFPAD_RCL[0])
MFPAD_RCL[0].T.shape

In [ ]:
fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row')
fig.subplots_adjust(hspace = .5, wspace=.5)

cosphi=np.array(cosphi)

# WHATCH OUT! experimental data are saved in the form (cos(theta), phi) looping internally on cos(theta) -> subplots has to be filled column by column instead of row by row
for i,ax in zip(range(72),axes.T.flatten()) :
    ax.contourf(cosphi[:,1].reshape(12,6),cosphi[:,0].reshape(12,6),MFPAD_RCL[i])
    
fig.tight_layout()
plt.show()